In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
import pandas as pd
from src.fetch import fetch_sources
from src.preprocess import preprocess
from src.utils import df_na_vals

In [4]:
df = fetch_sources()
df
# preprocess(raw)

False


,bw_nvlink,compute_cap,cpu_cores,cpu_name,cpu_ram,credit_discount_max,cuda_max_good,direct_port_count,disk_bw,disk_name,...,pcie_bw,public_ipaddr,reliability,score,static_ip,storage_cost,total_flops,verification,timestamp,country
0,0,860,56,Xeon® E5-2680 v4,126,400,12.2,400,1867,SK,...,112,39.165.212.208,8476,20,0,150,48,0,1702013330,CN
1,0,860,8,Xeon® E5-1630 v4,31,400,12.0,0,974,FIKWOT,...,116,107.185.207.235,6000,12,0,150,40,0,1702013330,US
2,0,860,88,Xeon® E5-2699 v4,220,400,12.3,1001,515,SATA,...,56,206.125.129.212,6064,3,1,150,206,0,1702013330,US
3,0,890,12,Core™ i7-7800X,251,400,12.0,0,1745,nvme,...,59,107.185.207.235,5826,8,0,150,163,0,1702013330,US
4,0,890,32,AMD EPYC 73F3 16-Core Processor,252,400,12.2,1001,2926,nvme,...,244,45.142.211.7,6398,37,0,10,163,0,1702013330,DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801,0,860,12,Core™ i7-7800X,125,200,12.2,100,2442,nvme,...,117,81.166.162.13,9898,108,1,200,141,1,1702013330,NO
2802,0,610,48,Xeon® E5-2695 v2,126,400,12.0,100,1282,KINGSTON,...,57,142.112.39.215,9814,60,1,200,114,1,1702013330,CA
2803,0,610,32,Xeon® E5-2650 v2,63,100,12.0,1000,1568,PNY,...,58,194.44.114.10,9963,46,1,150,49,2,1702013330,UA
2804,0,750,32,AMD Ryzen Threadripper 1950X 16-Core Processor,63,400,12.2,1000,3132,nvme,...,63,23.245.210.159,9973,36,1,150,60,1,1702013330,US


In [5]:
cpu_name = df.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = df.cpu_name.str.replace('Core™', '')
df.cpu_name = cpu_name

df['cores'] = (df.cpu_cores / df.num_gpus).fillna(0).astype(int)
df['ram'] = (df.cpu_ram / df.num_gpus).round().astype(int)
df['dph'] = (df.dph_base / df.num_gpus).round().astype(int)
df['disk'] = (df.disk_space / df.num_gpus).astype(int)
df['rented_fraq'] = '[' + df.num_gpus_rented.astype(str) + "/" + df.num_gpus.astype(str) + ']'
# df['rented'] = df.num_gpus_rented > 0
df['rented'] = df.num_gpus_rented == df.num_gpus
# df['min_fraq'] = df.min_chunk/df.num_gpus

# df.dlperf = df.dlperf.round(1)
# df.dlperf_per_dphtotal = df.dlperf_per_dphtotal.round(1)
# df.disk_bw = df.disk_bw.round()
# df.duration = pd.to_timedelta(df.duration * 1e9).dt.days
# df.inet_down = df.inet_down.fillna(0).astype(int)
# df.inet_up = df.inet_up.fillna(0).astype(int)
# df.inet_up_cost = (df.inet_up_cost * 1000).fillna(0).astype(int)
# df.inet_down_cost = (df.inet_down_cost * 1000).fillna(0).astype(int)

# df['isp'] = df.location.apply(lambda x: x.get('isp', None) if not pd.isna(x) else None)

# df.end_date = (pd.to_datetime(df.end_date * 1e9) - pd.Timestamp.now()).dt.days
df.end_date = pd.to_datetime(df.end_date * 1e9)
# df.start_date = pd.to_datetime(df.start_date * 1e9)
# df.set_index('machine_id', inplace=True)

In [6]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability', 'inet_up', 'inet_down', 'inet_up_cost', 'inet_down_cost', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'country']

df.query("dph >= 0.5 and gpu_name == 'RTX 4090'")[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
877,RTX 4090,[0/2],2000,400,16,126,3205,3246,1,US,...,1,12.2,535.129.03,4,16,0,245,AMD EPYC 7302P 16C Processor,ROMED8,US
891,RTX 4090,[0/2],1000,400,16,62,205,5335,1,MD,...,0,12.0,525.147.05,4,8,0,127,AMD Ryzen 9 7950X 16C Processor,X670E Taichi,MD
1911,RTX 4090,[1/1],900,0,64,252,1370,4461,1,JP,...,20,12.0,525.125.06,4,16,0,238,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,JP
1951,RTX 4090,[4/4],897,400,32,32,1667,6891,1,US,...,0,12.2,535.129.03,4,8,0,127,AMD EPYC 7B12 64C Processor,ROMED8,US
2742,RTX 4090,[8/8],850,200,24,63,102,3779,8,CA,...,3,12.2,535.104.05,4,16,1,232,AMD EPYC 7643 48C Processor,ROME2D32GM,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,RTX 4090,[0/2],265,0,12,32,410,2881,2,CZ,...,0,12.2,535.129.03,4,8,0,128,AMD Ryzen 9 5900X 12C Processor,ProArt B550,CZ
870,RTX 4090,[2/2],250,400,12,32,810,2714,1,KR,...,2,12.2,535.129.03,3,16,0,114,Xeon® E5-2650 v4,E5,KR
1806,RTX 4090,[1/1],206,300,24,94,1270,6407,1,CA,...,0,12.2,535.129.03,4,16,0,228,13th Gen Core™ i7-13700K,Z790 Pro RS,CA
172,RTX 4090,[7/7],200,400,9,36,235,975,7,UA,...,2,12.0,525.147.05,4,16,0,242,AMD Ryzen TR PRO 5975WX 32Cs,Pro WS WRX80E,UA


In [9]:
df.query('num_gpus == 8')[cols]

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
8,RTX 3080 Ti,[0/8],149,400,4,4,97,4182,8,KZ,...,2,12.2,535.113.01,4,16,0,231,13th Gen Core™ i9-13900K,PRIME Z790,KZ
22,RTX 3060,[8/8],60,400,8,32,717,1851,2,CN,...,2,12.2,535.129.03,3,16,0,109,Xeon® E5-2682 v4,X10DRG,CN
31,RTX 3060,[0/8],60,400,8,32,720,1768,2,CN,...,2,12.2,535.129.03,3,16,0,113,Xeon® E5-2682 v4,X10DRG,CN
34,RTX 3060,[0/8],60,400,8,32,720,1871,4,CN,...,2,12.2,535.129.03,3,16,0,108,Xeon® E5-2682 v4,X10DRG,CN
35,RTX 3060,[0/8],60,400,8,32,720,1842,1,CN,...,2,12.2,535.129.03,3,16,0,109,Xeon® E5-2682 v4,X10DRG,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,RTX 3090,[6/8],250,200,8,32,102,2467,1,NO,...,20,12.2,535.54.03,4,8,0,249,AMD Ryzen TR PRO 3975WX 32Cs,Pro WS WRX80E,NO
2796,RTX 4090,[8/8],725,100,32,63,607,380,8,CA,...,3,12.0,525.105.17,4,16,1,228,AMD EPYC 7B13 64C Processor,ROME2D32GM,CA
2797,RTX 4090,[8/8],500,200,8,32,41,2940,1,NO,...,20,12.2,535.129.03,4,16,0,243,AMD Ryzen TR PRO 5975WX 32Cs,Pro WS WRX80E,NO
2800,RTX 4090,[4/8],500,200,16,32,83,3006,1,NO,...,20,12.2,535.129.03,4,16,0,128,AMD Ryzen TR PRO 3995WX 64Cs,Pro WS WRX80E,NO


In [8]:
df[cols].query('country == "RU"')

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country
59,RTX 3090,[1/1],280,400,56,94,740,1775,1,RU,...,0,12.2,535.129.03,3,16,0,115,Xeon® E5-2680 v4,X99 DUAL Z8,RU
92,RTX 4090,[0/2],396,400,16,63,1670,5770,2,RU,...,1,12.2,535.129.03,4,8,0,128,AMD Ryzen 9 5950X 16C Processor,PRIME X570,RU
266,RTX 4090,[0/2],496,400,16,63,1670,5294,2,RU,...,2,12.2,535.129.03,4,8,0,128,AMD Ryzen 9 5950X 16C Processor,ROG STRIX X570,RU
357,RTX 4090,[2/2],149,400,6,16,5,427,1,RU,...,1,12.2,535.129.03,3,4,0,27,AMD Ryzen 5 3600 6C Processor,B550M AORUS ELITE,RU
435,RTX 4090,[0/2],396,400,16,63,1670,5533,2,RU,...,1,12.2,535.129.03,4,8,0,128,AMD Ryzen 9 5950X 16C Processor,PRIME X570,RU
570,RTX 4090,[0/3],420,100,21,63,546,1949,1,RU,...,0,12.2,535.129.03,3,16,0,125,AMD Ryzen TR 3970X 32C Processor,PRIME TRX40,RU
802,RTX 4090,[2/2],298,400,6,16,315,1793,1,RU,...,1,12.2,535.129.03,4,8,0,127,AMD Ryzen 5 5600 6C Processor,X570 AORUS MASTER,RU
806,RTX 3090,[3/3],149,400,4,10,113,1509,1,RU,...,1,12.2,535.129.03,3,2,0,14,AMD Ryzen 5 5600 6C Processor,B550 AORUS ELITE V2,RU
807,RTX 4090,[5/5],450,100,12,50,530,3086,1,RU,...,0,12.2,535.129.03,4,16,0,169,AMD Ryzen TR PRO 3975WX 32Cs,WRX80 Creator R2,RU
831,RTX 4090,[1/1],396,50,32,63,3240,5377,1,RU,...,1,12.2,535.129.03,4,16,0,250,AMD Ryzen 9 5950X 16C Processor,PRIME X570,RU
